In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-06 19:50:34.342010: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-06 19:50:34.825949: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-06 19:50:34.825963: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-06 19:50:34.865978: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-06 19:50:35.866892: W tensorflow/stream_executor/platform/de

In [2]:
list_f=['AtomPairFP','Autocorr','AvalonFP','Charge','Connectivity','Constitution','Estate','EstateFP','Fragment','InfoContent',
       'Kappa','MACCSFP','MAP4','Matrix','MHFP6','MOE','MorganFP','Path','PharmacoErGFP','PharmacoPFP','Property','PubChemFP','RDkitFP','Topology','TorsionFP']
name_all = []
auc_roc_test=[]
for feature in list_f:
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    cl = df.columns.tolist()
    feat = cl[2:]
    leng = len(feat)
    lab_ta_fea = ['smiles','active']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    lab_ta_fea.remove('smiles')
    lab_ta_fea.remove('active')
    df.to_csv("main_set_plus_"+feature+"_xgb_columns.csv",index=False)
    read_c = pd.read_csv("main_set_plus_"+feature+"_xgb_columns.csv")
    df_main = pd.DataFrame(read_c)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_main.to_csv('xgb_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset = loader.create_dataset('xgb_'+feature+'.csv')


    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    cl_e = df_e.columns.tolist()
    feat_e = cl_e[2:]
    leng_e = len(feat_e)
    lab_ta_fea_e = ['smiles','active']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    lab_ta_fea_e.remove('smiles')
    lab_ta_fea_e.remove('active')
    df_e.to_csv("external_set_plus_"+feature+"_xgb_columns.csv",index=False)
    read_c_e = pd.read_csv("external_set_plus_"+feature+"_xgb_columns.csv")
    df_external = pd.DataFrame(read_c_e)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea_e)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_external.to_csv('external_xgb_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset_e = loader.create_dataset('external_xgb_'+feature+'.csv')
    
     
    X, y, w = dataset.X, dataset.y, dataset.w
    X_e, y_e, w_e = dataset_e.X, dataset_e.y, dataset_e.w
    
    xgboost_model = xgboost.XGBClassifier(
      max_depth=5,
      learning_rate=0.05,
      n_estimators=3000,
      gamma=0.0,
      min_child_weight=5,
      max_delta_step=1,
      subsample=0.53,
      colsample_bytree=0.66,
      colsample_bylevel=1,
      reg_alpha=0,
      reg_lambda=1,
      scale_pos_weight=1,
      base_score=0.5,
      seed=2017)
    
    xgboost_model.fit(X, y)

    
    proba = xgboost_model.predict_proba(X)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_main=pd.DataFrame()
    df_main['pred']=pro
    df_main.to_csv('pred_xgb_main_set_'+feature+'.csv',index=False)
 

    proba = xgboost_model.predict_proba(X_e)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_external=pd.DataFrame()
    df_external['pred']=pro
    df_external.to_csv('pred_xgb_external_set_'+feature+'.csv',index=False)
    
    y_l=[]
    for y in y_e:
        y = int(y)
        y_l.append(y)
    auc_roc=pd.DataFrame()
    auc_roc['label'] = y_l
    auc_roc['pred'] = pro
    y_label = auc_roc['label']
    y_pred = auc_roc['pred']
    y_label = np.array(y_label)
    y_pred = np.array(y_pred)    
    fpr = dict()
    tpr = dict() 
    roc_auc = dict()
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)
    name = 'xgb_'+feature
    name_all.append(name)
    globals()[name] = auc(fpr, tpr)
    auc_roc_test.append(globals()[name])
    xgboost_model=''

In [3]:
for feature in list_f:
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    cl = df.columns.tolist()
    feat = cl[2:]
    leng = len(feat)
    lab_ta_fea = ['smiles','active']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    lab_ta_fea.remove('smiles')
    lab_ta_fea.remove('active')
    df.to_csv("main_set_plus_"+feature+"_rf_columns.csv",index=False)
    read_c = pd.read_csv("main_set_plus_"+feature+"_rf_columns.csv")
    df_main = pd.DataFrame(read_c)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_main.to_csv('rf_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset = loader.create_dataset('rf_'+feature+'.csv')


    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    cl_e = df_e.columns.tolist()
    feat_e = cl_e[2:]
    leng_e = len(feat_e)
    lab_ta_fea_e = ['smiles','active']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    lab_ta_fea_e.remove('smiles')
    lab_ta_fea_e.remove('active')
    df_e.to_csv("external_set_plus_"+feature+"_rf_columns.csv",index=False)
    read_c_e = pd.read_csv("external_set_plus_"+feature+"_rf_columns.csv")
    df_external = pd.DataFrame(read_c_e)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea_e)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_external.to_csv('external_rf_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset_e = loader.create_dataset('external_rf_'+feature+'.csv')
    
     
    X, y, w = dataset.X, dataset.y, dataset.w
    X_e, y_e, w_e = dataset_e.X, dataset_e.y, dataset_e.w
    
    model=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_samples_leaf=1,min_samples_split=2,
            min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
    
    model.fit(X,y)
    
    proba = model.predict_proba(X)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_main=pd.DataFrame()
    df_main['pred']=pro
    df_main.to_csv('pred_rf_main_set_'+feature+'.csv',index=False)
        

    proba = model.predict_proba(X_e)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_external=pd.DataFrame()
    df_external['pred']=pro
    df_external.to_csv('pred_rf_external_set_'+feature+'.csv',index=False)

    y_l=[]
    for y in y_e:
        y = int(y)
        y_l.append(y)
    auc_roc=pd.DataFrame()
    auc_roc['label'] = y_l
    auc_roc['pred'] = pro
    y_label = auc_roc['label']
    y_pred = auc_roc['pred']
    y_label = np.array(y_label)
    y_pred = np.array(y_pred)    
    fpr = dict()
    tpr = dict() 
    roc_auc = dict()
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)
    name = 'rf_'+feature
    name_all.append(name)
    globals()[name] = auc(fpr, tpr)
    auc_roc_test.append(globals()[name])
    model=''

/tmp/ipykernel_1807578/1112436580.py:67: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X,y)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/tmp/ipykernel_1807578/1112436580.py:67: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X,y)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep

/tmp/ipykernel_1807578/1112436580.py:67: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X,y)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/tmp/ipykernel_1807578/1112436580.py:67: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X,y)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep

In [4]:
for feature in list_f:
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    a = df['active']
    c=[]
    for b in a:
        if b==1:
            b=0
        else:
            b=1
        c.append(b)

    df.insert(loc=2, column='inactive', value=c)
    cl = df.columns.tolist()
    feat = cl[3:]
    leng = len(feat)
    lab_ta_fea = ['smiles','active','inactive']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    lab_ta_fea.remove('smiles')
    lab_ta_fea.remove('active')
    lab_ta_fea.remove('inactive')
    df.to_csv("main_set_plus_"+feature+"_fcnn_columns.csv",index=False)
    read_c = pd.read_csv("main_set_plus_"+feature+"_fcnn_columns.csv")
    df_main = pd.DataFrame(read_c)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_main.to_csv('fcnn_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active","inactive"], id_field="smiles",

                                 featurizer=featurizer)

      dataset = loader.create_dataset('fcnn_'+feature+'.csv')


    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    a1 = df_e['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)
    df_e.insert(loc=2, column='inactive', value=c1)
    cl_e = df_e.columns.tolist()
    feat_e = cl_e[3:]
    leng_e = len(feat_e)
    lab_ta_fea_e = ['smiles','active','inactive']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    lab_ta_fea_e.remove('smiles')
    lab_ta_fea_e.remove('active')
    lab_ta_fea_e.remove('inactive')
    df_e.to_csv("external_set_plus_"+feature+"_fcnn_columns.csv",index=False)
    read_c_e = pd.read_csv("external_set_plus_"+feature+"_fcnn_columns.csv")
    df_external = pd.DataFrame(read_c_e)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea_e)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_external.to_csv('external_fcnn_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active","inactive"], id_field="smiles",

                                 featurizer=featurizer)

      dataset_e = loader.create_dataset('external_fcnn_'+feature+'.csv')
    
    X, y, w = dataset.X, dataset.y, dataset.w
    X_e, y_e, w_e = dataset_e.X, dataset_e.y, dataset_e.w

    
    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(feat),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X, y,
              batch_size=batch_size,
              epochs=100)

    score = model.evaluate(X_e, y_e, verbose=0)
    
    proba=model.predict(X,
                      batch_size=None,
                      verbose="auto",
                      steps=None,
                      callbacks=None,
                      max_queue_size=10,
                      workers=1,
                      use_multiprocessing=False)
    pro=[]
    for p in proba:
        pi = p[0]
        pro.append(pi)
    df_main=pd.DataFrame()
    df_main['pred']=pro
    df_main.to_csv('pred_fcnn_main_set_'+feature+'.csv',index=False)

        
    proba=model.predict(X_e,
                      batch_size=None,
                      verbose="auto",
                      steps=None,
                      callbacks=None,
                      max_queue_size=10,
                      workers=1,
                      use_multiprocessing=False)
    pro=[]
    for p in proba:
        pi = p[0]
        pro.append(pi)
    df_external=pd.DataFrame()
    df_external['pred']=pro
    df_external.to_csv('pred_fcnn_external_set_'+feature+'.csv',index=False)
    
    name = 'fcnn_'+feature
    name_all.append(name)
    globals()[name] = score[1]
    auc_roc_test.append(globals()[name])
    model=''

2022-11-06 20:11:40.649479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 20:11:40.650135: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-06 20:11:40.650190: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-06 20:11:40.650237: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-06 20:11:40.650285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

Epoch 1/100
1/1 [==============================] - 1s 579ms/step - loss: 66.2742 - auc: 0.7459
Epoch 2/100
1/1 [==============================] - 0s 101ms/step - loss: 61.7706 - auc: 0.8274
Epoch 3/100
1/1 [==============================] - 0s 96ms/step - loss: 56.6957 - auc: 0.8319
Epoch 4/100
1/1 [==============================] - 0s 101ms/step - loss: 52.9968 - auc: 0.4827
Epoch 5/100
1/1 [==============================] - 0s 101ms/step - loss: 48.5481 - auc: 0.8592
Epoch 6/100
1/1 [==============================] - 0s 101ms/step - loss: 45.1183 - auc: 0.8772
Epoch 7/100
1/1 [==============================] - 0s 100ms/step - loss: 41.7969 - auc: 0.8798
Epoch 8/100
1/1 [==============================] - 0s 101ms/step - loss: 38.5558 - auc: 0.8830
Epoch 9/100
1/1 [==============================] - 0s 99ms/step - loss: 35.4617 - auc: 0.8880
Epoch 10/100
1/1 [==============================] - 0s 105ms/step - loss: 32.6859 - auc: 0.8660
Epoch 11/100
1/1 [==============================] -

1/1 [==============================] - 0s 46ms/step - loss: 17.6161 - auc: 0.7687
Epoch 72/100
1/1 [==============================] - 0s 45ms/step - loss: 15.5113 - auc: 0.7903
Epoch 73/100
1/1 [==============================] - 0s 43ms/step - loss: 18.5510 - auc: 0.6965
Epoch 74/100
1/1 [==============================] - 0s 45ms/step - loss: 22.2613 - auc: 0.8042
Epoch 75/100
1/1 [==============================] - 0s 45ms/step - loss: 18.2731 - auc: 0.8078
Epoch 76/100
1/1 [==============================] - 0s 45ms/step - loss: 12.1125 - auc: 0.8036
Epoch 77/100
1/1 [==============================] - 0s 53ms/step - loss: 17.4557 - auc: 0.5822
Epoch 78/100
1/1 [==============================] - 0s 49ms/step - loss: 14.8147 - auc: 0.8019
Epoch 79/100
1/1 [==============================] - 0s 54ms/step - loss: 10.5639 - auc: 0.8063
Epoch 80/100
1/1 [==============================] - 0s 47ms/step - loss: 9.8281 - auc: 0.8070
Epoch 81/100
1/1 [==============================] - 0s 47ms/step

1/1 [==============================] - 0s 29ms/step - loss: 0.7605 - auc: 0.7497
Epoch 42/100
1/1 [==============================] - 0s 26ms/step - loss: 0.7576 - auc: 0.7500
Epoch 43/100
1/1 [==============================] - 0s 27ms/step - loss: 0.7452 - auc: 0.7600
Epoch 44/100
1/1 [==============================] - 0s 26ms/step - loss: 0.7486 - auc: 0.7635
Epoch 45/100
1/1 [==============================] - 0s 23ms/step - loss: 0.7373 - auc: 0.7675
Epoch 46/100
1/1 [==============================] - 0s 24ms/step - loss: 0.7345 - auc: 0.7693
Epoch 47/100
1/1 [==============================] - 0s 28ms/step - loss: 0.7434 - auc: 0.7704
Epoch 48/100
1/1 [==============================] - 0s 25ms/step - loss: 0.7391 - auc: 0.7626
Epoch 49/100
1/1 [==============================] - 0s 25ms/step - loss: 0.7255 - auc: 0.7661
Epoch 50/100
1/1 [==============================] - 0s 27ms/step - loss: 0.7395 - auc: 0.7530
Epoch 51/100
1/1 [==============================] - 0s 26ms/step - loss: 

1/1 [==============================] - 0s 32ms/step - loss: 2.5587 - auc: 0.8156
Epoch 14/100
1/1 [==============================] - 0s 27ms/step - loss: 2.4719 - auc: 0.8151
Epoch 15/100
1/1 [==============================] - 0s 27ms/step - loss: 2.2998 - auc: 0.8170
Epoch 16/100
1/1 [==============================] - 0s 33ms/step - loss: 2.0494 - auc: 0.8088
Epoch 17/100
1/1 [==============================] - 0s 24ms/step - loss: 1.9736 - auc: 0.7593
Epoch 18/100
1/1 [==============================] - 0s 32ms/step - loss: 2.0840 - auc: 0.6698
Epoch 19/100
1/1 [==============================] - 0s 26ms/step - loss: 2.0440 - auc: 0.6759
Epoch 20/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8422 - auc: 0.7532
Epoch 21/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8050 - auc: 0.7961
Epoch 22/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8171 - auc: 0.8182
Epoch 23/100
1/1 [==============================] - 0s 27ms/step - loss: 

...

1/1 [==============================] - 0s 41ms/step - loss: 0.5850 - auc: 0.8549
Epoch 87/100
1/1 [==============================] - 0s 46ms/step - loss: 0.5777 - auc: 0.8603
Epoch 88/100
1/1 [==============================] - 0s 46ms/step - loss: 0.5887 - auc: 0.8517
Epoch 89/100
1/1 [==============================] - 0s 46ms/step - loss: 0.5926 - auc: 0.8512
Epoch 90/100
1/1 [==============================] - 0s 40ms/step - loss: 0.5957 - auc: 0.8514
Epoch 91/100
1/1 [==============================] - 0s 43ms/step - loss: 0.6026 - auc: 0.8474
Epoch 92/100
1/1 [==============================] - 0s 39ms/step - loss: 0.5872 - auc: 0.8530
Epoch 93/100
1/1 [==============================] - 0s 38ms/step - loss: 0.5911 - auc: 0.8489
Epoch 94/100
1/1 [==============================] - 0s 39ms/step - loss: 0.5747 - auc: 0.8571
Epoch 95/100
1/1 [==============================] - 0s 41ms/step - loss: 0.5752 - auc: 0.8571
Epoch 96/100
1/1 [==============================] - 0s 41ms/step - loss: 

1/1 [==============================] - 0s 23ms/step - loss: 2.8338 - auc: 0.7299
Epoch 59/100
1/1 [==============================] - 0s 24ms/step - loss: 2.6505 - auc: 0.7256
Epoch 60/100
1/1 [==============================] - 0s 25ms/step - loss: 2.7774 - auc: 0.7293
Epoch 61/100
1/1 [==============================] - 0s 23ms/step - loss: 2.5705 - auc: 0.7403
Epoch 62/100
1/1 [==============================] - 0s 25ms/step - loss: 2.5276 - auc: 0.7144
Epoch 63/100
1/1 [==============================] - 0s 27ms/step - loss: 2.4378 - auc: 0.7395
Epoch 64/100
1/1 [==============================] - 0s 25ms/step - loss: 2.4214 - auc: 0.7456
Epoch 65/100
1/1 [==============================] - 0s 29ms/step - loss: 2.3883 - auc: 0.7074
Epoch 66/100
1/1 [==============================] - 0s 28ms/step - loss: 2.2681 - auc: 0.7518
Epoch 67/100
1/1 [==============================] - 0s 28ms/step - loss: 2.1771 - auc: 0.7505
Epoch 68/100
1/1 [==============================] - 0s 24ms/step - loss: 

1/1 [==============================] - 0s 53ms/step - loss: 2.0492 - auc: 0.8688
Epoch 45/100
1/1 [==============================] - 0s 61ms/step - loss: 1.9492 - auc: 0.8648
Epoch 46/100
1/1 [==============================] - 0s 57ms/step - loss: 1.9112 - auc: 0.8384
Epoch 47/100
1/1 [==============================] - 0s 58ms/step - loss: 1.7910 - auc: 0.8566
Epoch 48/100
1/1 [==============================] - 0s 54ms/step - loss: 1.7451 - auc: 0.8479
Epoch 49/100
1/1 [==============================] - 0s 54ms/step - loss: 1.6555 - auc: 0.8681
Epoch 50/100
1/1 [==============================] - 0s 55ms/step - loss: 1.6144 - auc: 0.8660
Epoch 51/100
1/1 [==============================] - 0s 56ms/step - loss: 1.5344 - auc: 0.8767
Epoch 52/100
1/1 [==============================] - 0s 57ms/step - loss: 1.5008 - auc: 0.8662
Epoch 53/100
1/1 [==============================] - 0s 60ms/step - loss: 1.4325 - auc: 0.8677
Epoch 54/100
1/1 [==============================] - 0s 58ms/step - loss: 

1/1 [==============================] - 0s 30ms/step - loss: 1.3904 - auc: 0.8132
Epoch 15/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3343 - auc: 0.8050
Epoch 16/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2391 - auc: 0.8003
Epoch 17/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2443 - auc: 0.7607
Epoch 18/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2336 - auc: 0.7297
Epoch 19/100
1/1 [==============================] - 0s 24ms/step - loss: 1.2274 - auc: 0.7240
Epoch 20/100
1/1 [==============================] - 0s 25ms/step - loss: 1.2008 - auc: 0.7375
Epoch 21/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1482 - auc: 0.7776
Epoch 22/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1086 - auc: 0.8073
Epoch 23/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1591 - auc: 0.8054
Epoch 24/100
1/1 [==============================] - 0s 27ms/step - loss: 

1/1 [==============================] - 0s 108ms/step - loss: 0.6268 - auc: 0.8532
Epoch 87/100
1/1 [==============================] - 0s 99ms/step - loss: 0.6119 - auc: 0.8562
Epoch 88/100
1/1 [==============================] - 0s 99ms/step - loss: 0.6144 - auc: 0.8542
Epoch 89/100
1/1 [==============================] - 0s 100ms/step - loss: 0.6160 - auc: 0.8501
Epoch 90/100
1/1 [==============================] - 0s 101ms/step - loss: 0.6084 - auc: 0.8528
Epoch 91/100
1/1 [==============================] - 0s 101ms/step - loss: 0.6220 - auc: 0.8483
Epoch 92/100
1/1 [==============================] - 0s 109ms/step - loss: 0.6162 - auc: 0.8529
Epoch 93/100
1/1 [==============================] - 0s 100ms/step - loss: 0.5930 - auc: 0.8609
Epoch 94/100
1/1 [==============================] - 0s 95ms/step - loss: 0.5916 - auc: 0.8604
Epoch 95/100
1/1 [==============================] - 0s 98ms/step - loss: 0.5878 - auc: 0.8599
Epoch 96/100
1/1 [==============================] - 0s 92ms/step - 

In [5]:
for feature in list_f:
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    cl = df.columns.tolist()
    feat = cl[2:]
    leng = len(feat)
    lab_ta_fea = ['smiles','active']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    lab_ta_fea.remove('smiles')
    lab_ta_fea.remove('active')
    df.to_csv("main_set_plus_"+feature+"_lr_columns.csv",index=False)
    read_c = pd.read_csv("main_set_plus_"+feature+"_lr_columns.csv")
    df_main = pd.DataFrame(read_c)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_main.to_csv('lr_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset = loader.create_dataset('lr_'+feature+'.csv')


    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    cl_e = df_e.columns.tolist()
    feat_e = cl_e[2:]
    leng_e = len(feat_e)
    lab_ta_fea_e = ['smiles','active']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    lab_ta_fea_e.remove('smiles')
    lab_ta_fea_e.remove('active')
    df_e.to_csv("external_set_plus_"+feature+"_lr_columns.csv",index=False)
    read_c_e = pd.read_csv("external_set_plus_"+feature+"_lr_columns.csv")
    df_external = pd.DataFrame(read_c_e)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea_e)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_external.to_csv('external_lr_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset_e = loader.create_dataset('external_lr_'+feature+'.csv')
    sklearn_model = LogisticRegression(
          penalty= 'l2',
          C=1.0,
          class_weight="balanced",
          dual=False,
          fit_intercept=True,
          max_iter=100,
          multi_class='ovr',
          n_jobs=-1,
          random_state=None,solver='liblinear',tol=0.0001,verbose=0,warm_start=False)
    
    X, y, w = dataset.X, dataset.y, dataset.w
    X_e, y_e, w_e = dataset_e.X, dataset_e.y, dataset_e.w
    sklearn_model.fit(X, y)

    
    proba = sklearn_model.predict_proba(X)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_main=pd.DataFrame()
    df_main['pred']=pro
    df_main.to_csv('pred_lr_main_set_'+feature+'.csv',index=False)

        
    proba = sklearn_model.predict_proba(X_e)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_external=pd.DataFrame()
    df_external['pred']=pro
    df_external.to_csv('pred_lr_external_set_'+feature+'.csv',index=False)
    
    y_l=[]
    for y in y_e:
        y = int(y)
        y_l.append(y)
    auc_roc=pd.DataFrame()
    auc_roc['label'] = y_l
    auc_roc['pred'] = pro
    y_label = auc_roc['label']
    y_pred = auc_roc['pred']
    y_label = np.array(y_label)
    y_pred = np.array(y_pred)    
    fpr = dict()
    tpr = dict() 
    roc_auc = dict()
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)
    roc_auc = auc(fpr, tpr)
    name = 'lr_'+feature
    name_all.append(name)
    globals()[name] = auc(fpr, tpr)
    auc_roc_test.append(globals()[name])
    sklearn_model=''

/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'.

/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'.

In [6]:
for feature in list_f:
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    cl = df.columns.tolist()
    fea = cl[2:]
    leng = len(fea)
    lab_ta_fea = ['smiles','active']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    df2=pd.DataFrame()
    df2['smiles']=df['smiles']
    df2['active']=df['active']
    df2.to_csv('main_set_plus_'+feature+'_dmpnn_smile_active.csv',index=False)
    df.pop('smiles')
    df.pop('active')
    df.to_csv('main_set_plus_'+feature+'_dmpnn_feature.csv',index=False)


    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    cl_e = df_e.columns.tolist()
    fea_e = cl_e[2:]
    leng_e = len(fea_e)
    lab_ta_fea_e = ['smiles','active']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    df2_e=pd.DataFrame()
    df2_e['smiles']=df_e['smiles']
    df2_e['active']=df_e['active']
    label=df_e['active']
    df2_e.to_csv('external_set_plus_'+feature+'_dmpnn_smile_active.csv',index=False)
    df_e.pop('smiles')
    df_e.pop('active')
    df_e.to_csv('external_set_plus_'+feature+'_feature.csv',index=False)
    
    arguments = [
        '--data_path', 'main_set_plus_'+feature+'_dmpnn_smile_active.csv',
        '--features_only',
        '--features_path','main_set_plus_'+feature+'_dmpnn_feature.csv',
        '--dataset_type', 'classification',
        '--save_dir', 'dmpnn_'+feature+'_checkpoints',
    ]

    args = chemprop.args.TrainArgs().parse_args(arguments)
    mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

    arguments = [
        '--test_path', 'main_set_plus_'+feature+'_dmpnn_smile_active.csv',
        '--features_path','main_set_plus_'+feature+'_dmpnn_feature.csv',
        '--preds_path', 'dmpnn_'+feature+'_pred_main.csv',
        '--checkpoint_dir', 'dmpnn_'+feature+'_checkpoints'
    ]
    
    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)
    file = pd.read_csv('dmpnn_'+feature+'_pred_main.csv')
    pred_df = pd.DataFrame(file)
    pred=pd.DataFrame()
    pred['pred'] =pred_df['active']
    df_main=pd.DataFrame()
    df_main['pred']=pred
    df_main.to_csv('pred_dmpnn_main_set_'+feature+'.csv',index=False)
        
    
    arguments = [
        '--test_path', 'external_set_plus_'+feature+'_dmpnn_smile_active.csv',
        '--features_path','external_set_plus_'+feature+'_feature.csv',
        '--preds_path', 'dmpnn_'+feature+'_pred_external.csv',
        '--checkpoint_dir', 'dmpnn_'+feature+'_checkpoints'
    ]

    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)
    file = pd.read_csv('dmpnn_'+feature+'_pred_external.csv')
    pred_df = pd.DataFrame(file)
    pred=pd.DataFrame()
    pred['pred'] =pred_df['active']
    df_external=pd.DataFrame()
    df_external['pred']=pred
    df_external.to_csv('pred_dmpnn_external_set_'+feature+'.csv',index=False)
    
    y_label = label
    y_pred = pred['pred']
    y_label = np.array(y_label)
    y_pred = np.array(y_pred)    
    fpr = dict()
    tpr = dict() 
    roc_auc = dict()
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)
    roc_auc = auc(fpr, tpr)
    name = 'dmpnn_'+feature
    name_all.append(name)
    globals()[name] = auc(fpr, tpr)
    auc_roc_test.append(globals()[name])

Command line
python /home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/hasee2/.local/share/jupyter/runtime/kernel-ad59c5d3-8efa-410c-9c5d-323899d5183a.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'cuda': True,
 'data_path': 'main_set_plus_AtomPairFP_dmpnn_smile_active.csv',
 'data_weights_path': None,
 'dataset_

 89%|██████████████████████████████████████▍    | 50/56 [00:01<00:00, 41.08it/s]Loss = 1.1202e-01, PNorm = 23.7998, GNorm = 0.8059, lr_0 = 7.1433e-04

 98%|██████████████████████████████████████████▏| 55/56 [00:01<00:00, 41.97it/s]
                                                                                
 71%|████████████████████████████████▏            | 5/7 [00:00<00:00, 44.28it/s]
                                                                                Validation auc = 0.925994
 20%|████████▊                                   | 6/30 [00:14<00:44,  1.85s/it]Epoch 6

  9%|███▉                                        | 5/56 [00:00<00:01, 42.01it/s]Loss = 6.0307e-02, PNorm = 23.8212, GNorm = 1.0182, lr_0 = 7.0372e-04

 27%|███████████▌                               | 15/56 [00:00<00:01, 40.58it/s]Loss = 4.8647e-02, PNorm = 23.8417, GNorm = 0.7849, lr_0 = 6.9328e-04

 45%|███████████████████▏                       | 25/56 [00:00<00:00, 41.08it/s]Loss = 7.9134e-02, PNorm = 23


 71%|██████████████████████████████▋            | 40/56 [00:00<00:00, 42.11it/s]Loss = 3.6068e-02, PNorm = 24.3430, GNorm = 1.0697, lr_0 = 3.4077e-04

 89%|██████████████████████████████████████▍    | 50/56 [00:01<00:00, 43.61it/s]Loss = 3.3632e-02, PNorm = 24.3532, GNorm = 0.9872, lr_0 = 3.3572e-04

 98%|██████████████████████████████████████████▏| 55/56 [00:01<00:00, 43.66it/s]Loss = 1.3748e-01, PNorm = 24.3529, GNorm = 1.5043, lr_0 = 3.3522e-04

                                                                                
 71%|████████████████████████████████▏            | 5/7 [00:00<00:00, 44.45it/s]
                                                                                Validation auc = 0.929339
 50%|█████████████████████▌                     | 15/30 [00:28<00:23,  1.55s/it]Epoch 15

  9%|███▉                                        | 5/56 [00:00<00:01, 43.18it/s]Loss = 2.5938e-02, PNorm = 24.3555, GNorm = 0.1586, lr_0 = 3.3024e-04

 27%|███████████▌                    

 45%|███████████████████▏                       | 25/56 [00:00<00:00, 43.11it/s]Loss = 1.4894e-02, PNorm = 24.5396, GNorm = 0.0469, lr_0 = 1.6477e-04

 62%|██████████████████████████▉                | 35/56 [00:00<00:00, 43.25it/s]Loss = 3.0546e-02, PNorm = 24.5418, GNorm = 0.0778, lr_0 = 1.6233e-04

 80%|██████████████████████████████████▌        | 45/56 [00:01<00:00, 43.01it/s]Loss = 1.0860e-02, PNorm = 24.5444, GNorm = 0.1433, lr_0 = 1.5992e-04

 98%|██████████████████████████████████████████▏| 55/56 [00:01<00:00, 43.75it/s]
                                                                                
 71%|████████████████████████████████▏            | 5/7 [00:00<00:00, 44.51it/s]
                                                                                Validation auc = 0.929339
 80%|██████████████████████████████████▍        | 24/30 [00:42<00:08,  1.50s/it]Epoch 24

  0%|                                                    | 0/56 [00:00<?, ?it/s]Loss = 7.8880e-02, PNorm = 2


 98%|██████████████████████████████████████████▏| 55/56 [00:01<00:00, 41.18it/s]Loss = 4.8256e-03, PNorm = 24.5954, GNorm = 0.1192, lr_0 = 1.1287e-04

                                                                                
 71%|████████████████████████████████▏            | 5/7 [00:00<00:00, 44.77it/s]
                                                                                Validation auc = 0.929238
 93%|████████████████████████████████████████▏  | 28/30 [00:48<00:03,  1.53s/it]Epoch 28

  9%|███▉                                        | 5/56 [00:00<00:01, 44.93it/s]Loss = 1.5230e-02, PNorm = 24.5961, GNorm = 0.1917, lr_0 = 1.1120e-04

 27%|███████████▌                               | 15/56 [00:00<00:00, 43.22it/s]Loss = 1.8198e-02, PNorm = 24.5981, GNorm = 0.0431, lr_0 = 1.0955e-04

 45%|███████████████████▏                       | 25/56 [00:00<00:00, 43.40it/s]Loss = 2.1756e-02, PNorm = 24.5989, GNorm = 0.2723, lr_0 = 1.0792e-04

 62%|██████████████████████████▉     

...

Validating SMILES
Test size = 440


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.22it/s]

Saving predictions to dmpnn_TorsionFP_pred_external.csv
Elapsed time = 0:00:01


In [7]:
for feature in list_f:
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    cl = df.columns.tolist()
    feat = cl[2:]
    leng = len(feat)
    lab_ta_fea = ['smiles','active']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    lab_ta_fea.remove('smiles')
    lab_ta_fea.remove('active')
    df.to_csv("main_set_plus_"+feature+"_svm_columns.csv",index=False)
    read_c = pd.read_csv("main_set_plus_"+feature+"_svm_columns.csv")
    df_main = pd.DataFrame(read_c)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_main.to_csv('svm_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset = loader.create_dataset('svm_'+feature+'.csv')


    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    cl_e = df_e.columns.tolist()
    feat_e = cl_e[2:]
    leng_e = len(feat_e)
    lab_ta_fea_e = ['smiles','active']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    lab_ta_fea_e.remove('smiles')
    lab_ta_fea_e.remove('active')
    df_e.to_csv("external_set_plus_"+feature+"_svm_columns.csv",index=False)
    read_c_e = pd.read_csv("external_set_plus_"+feature+"_svm_columns.csv")
    df_external = pd.DataFrame(read_c_e)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea_e)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_external.to_csv('external_svm_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset_e = loader.create_dataset('external_svm_'+feature+'.csv')
    
     
    X, y, w = dataset.X, dataset.y, dataset.w
    X_e, y_e, w_e = dataset_e.X, dataset_e.y, dataset_e.w
    
    model=SVC(C=1,gamma=0.05,class_weight='balanced',probability=True)
    
    model.fit(X,y)
    
    proba = model.predict_proba(X)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_main=pd.DataFrame()
    df_main['pred']=pro
    df_main.to_csv('pred_svm_main_set_'+feature+'.csv',index=False)
        

    proba = model.predict_proba(X_e)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_external=pd.DataFrame()
    df_external['pred']=pro
    df_external.to_csv('pred_svm_external_set_'+feature+'.csv',index=False)

    y_l=[]
    for y in y_e:
        y = int(y)
        y_l.append(y)
    auc_roc=pd.DataFrame()
    auc_roc['label'] = y_l
    auc_roc['pred'] = pro
    y_label = auc_roc['label']
    y_pred = auc_roc['pred']
    y_label = np.array(y_label)
    y_pred = np.array(y_pred)    
    fpr = dict()
    tpr = dict() 
    roc_auc = dict()
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)
    name = 'svm_'+feature
    name_all.append(name)
    globals()[name] = auc(fpr, tpr)
    auc_roc_test.append(globals()[name])
    model=''

/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-v

In [8]:
print(name_all)
print(auc_roc_test)
de=pd.DataFrame()
de['name']=name_all
de['roc']=auc_roc_test
de.to_csv('single_model_auc.csv',index=False)

['xgb_AtomPairFP', 'xgb_Autocorr', 'xgb_AvalonFP', 'xgb_Charge', 'xgb_Connectivity', 'xgb_Constitution', 'xgb_Estate', 'xgb_EstateFP', 'xgb_Fragment', 'xgb_InfoContent', 'xgb_Kappa', 'xgb_MACCSFP', 'xgb_MAP4', 'xgb_Matrix', 'xgb_MHFP6', 'xgb_MOE', 'xgb_MorganFP', 'xgb_Path', 'xgb_PharmacoErGFP', 'xgb_PharmacoPFP', 'xgb_Property', 'xgb_PubChemFP', 'xgb_RDkitFP', 'xgb_Topology', 'xgb_TorsionFP', 'rf_AtomPairFP', 'rf_Autocorr', 'rf_AvalonFP', 'rf_Charge', 'rf_Connectivity', 'rf_Constitution', 'rf_Estate', 'rf_EstateFP', 'rf_Fragment', 'rf_InfoContent', 'rf_Kappa', 'rf_MACCSFP', 'rf_MAP4', 'rf_Matrix', 'rf_MHFP6', 'rf_MOE', 'rf_MorganFP', 'rf_Path', 'rf_PharmacoErGFP', 'rf_PharmacoPFP', 'rf_Property', 'rf_PubChemFP', 'rf_RDkitFP', 'rf_Topology', 'rf_TorsionFP', 'fcnn_AtomPairFP', 'fcnn_Autocorr', 'fcnn_AvalonFP', 'fcnn_Charge', 'fcnn_Connectivity', 'fcnn_Constitution', 'fcnn_Estate', 'fcnn_EstateFP', 'fcnn_Fragment', 'fcnn_InfoContent', 'fcnn_Kappa', 'fcnn_MACCSFP', 'fcnn_MAP4', 'fcnn_Matr

In [9]:
list_m=['xgb','fcnn','lr','rf','svm']
list_f=['AtomPairFP','Autocorr','AvalonFP','Charge','Connectivity','Constitution','Estate','EstateFP','Fragment','InfoContent',
       'Kappa','MACCSFP','MAP4','Matrix','MHFP6','MOE','MorganFP','Path','PharmacoErGFP','PharmacoPFP','Property','PubChemFP','RDkitFP','Topology','TorsionFP']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        if os.path.exists('main_set_plus_'+feature+'_'+m+'_columns.csv'):
            os.remove('main_set_plus_'+feature+'_'+m+'_columns.csv')
        if os.path.exists('external_set_plus_'+feature+'_'+m+'_columns.csv'):
            os.remove('external_set_plus_'+feature+'_'+m+'_columns.csv')
        if os.path.exists(m+'_'+feature+'.csv'):
            os.remove(m+'_'+feature+'.csv')
        if os.path.exists('external_'+m+'_'+feature+'.csv'):
            os.remove('external_'+m+'_'+feature+'.csv')
        if os.path.exists('pred_'+m+'__main_set_'+feature+'.csv'):
            os.remove('pred_'+m+'__main_set_'+feature+'.csv')
        if os.path.exists('pred_'+m+'__external_set_'+feature+'.csv'):
            os.remove('pred_'+m+'__external_set_'+feature+'.csv')
        if os.path.exists('pred_'+m+'__pred_set_'+feature+'.csv'):
            os.remove('pred_'+m+'__pred_set_'+feature+'.csv')


In [10]:
list_f=['AtomPairFP','Autocorr','AvalonFP','Charge','Connectivity','Constitution','Estate','EstateFP','Fragment','InfoContent',
       'Kappa','MACCSFP','MAP4','Matrix','MHFP6','MOE','MorganFP','Path','PharmacoErGFP','PharmacoPFP','Property','PubChemFP','RDkitFP','Topology','TorsionFP']
for feature in list_f:
    if os.path.exists('main_set_plus_'+feature+'_dmpnn_smile_active.csv'):
        os.remove('main_set_plus_'+feature+'_dmpnn_smile_active.csv')
    if os.path.exists('main_set_plus_'+feature+'_dmpnn_feature.csv'):
        os.remove('main_set_plus_'+feature+'_dmpnn_feature.csv')
    if os.path.exists('dmpnn_'+feature+'_pred_main.csv'):
        os.remove('dmpnn_'+feature+'_pred_main.csv')
    if os.path.exists('pred_set_plus_'+feature+'_dmpnn_smile_active.csv'):
        os.remove('pred_set_plus_'+feature+'_dmpnn_smile_active.csv')
    if os.path.exists('pred_set_plus_'+feature+'_dmpnn_feature.csv'):
        os.remove('pred_set_plus_'+feature+'_dmpnn_feature.csv')
    if os.path.exists('dmpnn_'+feature+'_pred_pred.csv'):
        os.remove('dmpnn_'+feature+'_pred_pred.csv')
    if os.path.exists('external_set_plus_'+feature+'_dmpnn_smile_active.csv'):
        os.remove('external_set_plus_'+feature+'_dmpnn_smile_active.csv')
    if os.path.exists('external_set_plus_'+feature+'_dmpnn_feature.csv'):
        os.remove('external_set_plus_'+feature+'_dmpnn_feature.csv')
    if os.path.exists('dmpnn_'+feature+'_pred_external.csv'):
        os.remove('dmpnn_'+feature+'_pred_external.csv')
    if os.path.exists('pred_set_plus_'+feature+'_dmpnn_smile.csv'):
        os.remove('pred_set_plus_'+feature+'_dmpnn_smile.csv')
    if os.path.exists('pred_set_plus_'+feature+'_feature.csv'):
        os.remove('pred_set_plus_'+feature+'_feature.csv')
    if os.path.exists('external_set_plus_'+feature+'_feature.csv'):
        os.remove('external_set_plus_'+feature+'_feature.csv')